In [ ]:
import sys
import functools
import numpy as np
from graphviz import Digraph, Source
from pathlib import Path
from pynini import Weight

sys.path.append(str(Path.home() / 'Code/Python/wynini'))
from wynini import config as wconfig
from wynini import wfst
from wynini.wfst import *

In [ ]:
help(wconfig)
wconfig.init({'sigma': ['a', 'b', 'c']})

In [ ]:
M = braid(2, arc_type='log')
one = Weight('log', 0.0)
M.assign_weights(lambda M, q, t: one)

#M = trellis(2)
#M = ngram(context='left', length=1)
M.draw('M.dot', show_weight_one=True)
M_viz = Source.from_file('M.dot')
M_viz

In [ ]:
beta = shortestdistance(M, reverse=True)
beta = np.array([float(w) for w in beta])
print(np.exp(-beta))

In [ ]:
# Add paths.
M.add_path(src=1, ilabel='a', olabel='b c', weight=None, dest=1)
M.add_path(src=2, ilabel='a b', olabel='c', weight=None, dest=3)
M.draw('M.dot', show_weight_one=True)
M_viz = Source.from_file('M.dot')
M_viz

In [ ]:
# Concatenation, union, optionality, star.
wfst1 = Wfst()
q0 = wfst1.add_state(initial=True)
q1 = wfst1.add_state(final=True)
wfst1.add_arc(q0, 'a', 'a', None, q1)
print(wfst1.print())

wfst2 = Wfst()
q0 = wfst2.add_state(initial=True)
q1 = wfst2.add_state(final=True)
wfst2.add_arc(q0, 'b', 'b', None, q1)
print(wfst2.print())

# Concatenation.
wfst12 = concat(wfst1, wfst2)
print(wfst12.print())

# Union.
wfst12 = union(wfst1, wfst2)
print(wfst12.print())

# Optionality.
wfst = ques(wfst1)
print(wfst.print())

# Star.
wfst = star(wfst1)
print(wfst.print())

In [ ]:
# N-gram machines.
config.init({'sigma': ['a', 'b']})
M = ngram(context='both', length=1)

# Flatten labels.
def relabel_func(wfst, q):
    label = wfst.state_label(q)
    label_ = tuple(' '.join(x) for x in label)
    return label_
M = M.relabel_states(relabel_func)

M.draw('M.dot', show_weight_one=False)
M_viz = Source.from_file('M.dot')
M_viz